# StackOverflow assistant bot

In this final project, we will combine everything we have learned about Natural Language Processing to construct a *dialogue chat bot*, which will be able to:
* answer programming-related questions (using StackOverflow dataset);
* chit-chat and simulate dialogue on all non programming-related questions.

For a chit-chat mode we will use a pre-trained neural network engine available from [ChatterBot](https://github.com/gunthercox/ChatterBot).
Those who aim at honor certificates for our course or are just curious, will train their own models for chit-chat.
![](https://imgs.xkcd.com/comics/twitter_bot.png)
©[xkcd](https://xkcd.com)

### Data description

To detect *intent* of users questions we will need two text collections:
- `tagged_posts.tsv` — StackOverflow posts, tagged with one programming language (*positive samples*).
- `dialogues.tsv` — dialogue phrases from movie subtitles (*negative samples*).


For those questions, that have programming-related intent, we will proceed as follow predict programming language (only one tag per question allowed here) and rank candidates within the tag using embeddings.
For the ranking part, you will need:
- `word_embeddings.tsv` — word embeddings, that you  trained with StarSpace in the 3rd assignment. It's not a problem if you didn't do it, because we can offer an alternative solution for you.

As a result of this notebook, you should obtain the following new objects that you will then use in the running bot:

- `intent_recognizer.pkl` — intent recognition model;
- `tag_classifier.pkl` — programming language classification model;
- `tfidf_vectorizer.pkl` — vectorizer used during training;
- `thread_embeddings_by_tags` — folder with thread embeddings, arranged by tags.
    

### Installing new modules
* pip3 install chatterbot==1.0.4
  - Had error in the installation and was solved by downgrading the verssion to 1.0.4 from the latest one 1.0.5

* pip3 install twitter
* pip3 install configparser


## Part I. Intent and language recognition

We want to write a bot, which will not only **answer programming-related questions**, but also will be able to **maintain a dialogue**. We would also like to detect the *intent* of the user from the question (we could have had a 'Question answering mode' check-box in the bot, but it wouldn't fun at all, would it?). So the first thing we need to do is to **distinguish programming-related questions from general ones**.

It would also be good to predict which programming language a particular question referees to. By doing so, we will speed up question search by a factor of the number of languages (10 here), and exercise our *text classification* skill a bit. :)

In [1]:
import numpy as np
import pandas as pd
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer

from utils import *

data_dir = "./datasets/"
pkl_dir = "./pickle_files/"


# Paths for all resources for the bot.
RESOURCE_PATH = { 
    'INTENT_RECOGNIZER': pkl_dir+'intent_recognizer.pkl',
    'TAG_CLASSIFIER': pkl_dir+'tag_classifier.pkl',
    'TFIDF_VECTORIZER': pkl_dir+'tfidf_vectorizer.pkl',
    'THREAD_EMBEDDINGS_FOLDER': pkl_dir+'thread_embeddings_by_tags',
    'WORD_EMBEDDINGS': data_dir+'word_embeddings.tsv',
}


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gshyam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Data preparation

In the first assignment (Predict tags on StackOverflow with linear models), you have already learnt how to preprocess texts and do TF-IDF tranformations. Reuse your code here. In addition, you will also need to [dump](https://docs.python.org/3/library/pickle.html#pickle.dump) the TF-IDF vectorizer with pickle to use it later in the running bot.

In [2]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
    vectorizer = TfidfVectorizer(min_df =5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
    X_train = vectorizer.fit_transform(X_train)
    X_test  = vectorizer.transform(X_test)
    with open(vectorizer_path, 'wb') as f:
        pickle.dump(vectorizer, f)
    return X_train, X_test

Now, load examples of two classes. Use a subsample of stackoverflow data to balance the classes. You will need the full data later.

In [3]:
sample_size = 20000  # 200000

dialogue_df      = pd.read_csv(data_dir+'dialogues.tsv',    sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv(data_dir+'tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

Check how the data look like:

In [4]:
print ("dialogue_df.head:")
display ( dialogue_df.head(2) )
print ("\nStackOverflow_df.head:")
display ( stackoverflow_df.head(2) )
print (f"dialogue_df.shape:{dialogue_df.shape} and stackoverflow_df.shape: {stackoverflow_df.shape}")

dialogue_df.head:


,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue



StackOverflow_df.head:


,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp


dialogue_df.shape:(20000, 2) and stackoverflow_df.shape: (20000, 3)


**text preparation**

In [5]:
from utils import text_prepare
dialogue_df['text'] = dialogue_df['text'].apply(text_prepare)
stackoverflow_df['title'] = stackoverflow_df['title'].apply(text_prepare)

### Intent recognition

We will do a binary classification on TF-IDF representations of texts. Labels will be either `dialogue` for general questions or `stackoverflow` for programming-related questions. First, prepare the data for this task:
- concatenate `dialogue` and `stackoverflow` examples into one sample
- split it into train and test in proportion 9:1, use *random_state=0* for reproducibility
- transform it into TF-IDF features

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
print (f"dialogue_df.text.shape: {dialogue_df['text'].shape}  \n\
stackoverflow_df.title.shape:{stackoverflow_df['title'].shape}")

dialogue_df.text.shape: (20000,)  
stackoverflow_df.title.shape:(20000,)


In [8]:

X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

print(f"X_Train.size = {len(X_train)}, X_test.size = {len(X_test)}")
print(f"y_Train.size = {len(y_train)}, y_test.size = {len(y_test)}")

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test, RESOURCE_PATH['TFIDF_VECTORIZER'])

X_Train.size = 36000, X_test.size = 4000
y_Train.size = 36000, y_test.size = 4000


Train the **intent recognizer** using LogisticRegression on the train set with the following parameters: *penalty='l2'*, *C=10*, *random_state=0*. Print out the accuracy on the test set to check whether everything looks good.

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
RESOURCE_PATH

{'INTENT_RECOGNIZER': './pickle_files/intent_recognizer.pkl',
 'TAG_CLASSIFIER': './pickle_files/tag_classifier.pkl',
 'TFIDF_VECTORIZER': './pickle_files/tfidf_vectorizer.pkl',
 'THREAD_EMBEDDINGS_FOLDER': './pickle_files/thread_embeddings_by_tags',
 'WORD_EMBEDDINGS': './datasets/word_embeddings.tsv'}

In [11]:
intent_recognizer = LogisticRegression(penalty='l2', C=10, random_state=0, solver='liblinear')
intent_recognizer.fit(X_train_tfidf, y_train)

LogisticRegression(C=10, random_state=0, solver='liblinear')

In [12]:
print ( X_train_tfidf.shape)
print ( X_test_tfidf.shape)
print ( len(y_train))

(36000, 7721)
(4000, 7721)
36000


In [13]:
intent_recognizer = LogisticRegression(penalty='l2', C=10, random_state=0, solver='liblinear')
intent_recognizer.fit(X_train_tfidf, y_train)
y_test_pred = intent_recognizer.predict(X_test_tfidf)

In [14]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test accuracy = {test_accuracy}")

Test accuracy = 0.98525


Dump the classifier to use it in the running bot.

In [15]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

### Programming language classification 

We will train one more classifier for the programming-related questions. It will predict exactly one tag (=programming language) and will be also based on Logistic Regression with TF-IDF features. 

First, let us prepare the data for this task.

In [16]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(f"X_Train.size = {len(X_train)},  \nX_test.size = {len(X_test)}")

X_Train.size = 16000,  
X_test.size = 4000


Let us reuse the TF-IDF vectorizer that we have already created above. It should not make a huge difference which data was used to train it.

In [18]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))
X_train_tfidf  = vectorizer.transform(X_train)
X_test_tfidf   = vectorizer.transform(X_test)

Train the **tag classifier** using OneVsRestClassifier wrapper over LogisticRegression. Use the following parameters: *penalty='l2'*, *C=5*, *random_state=0*.

In [19]:
from sklearn.multiclass import OneVsRestClassifier

In [20]:
tag_classifier = OneVsRestClassifier(LogisticRegression(penalty='l2', C=5, random_state=0, solver='liblinear'))
tag_classifier.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=5, random_state=0,
                                                 solver='liblinear'))

In [21]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.7425


Dump the classifier to use it in the running bot.

In [22]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

## Part II. Ranking  questions with embeddings

To find a relevant answer (a thread from StackOverflow) on a question you will use vector representations to calculate similarity between the question and existing threads. We already had `question_to_vec` function from the assignment 3, which can create such a representation based on word vectors. 

However, it would be costly to compute such a representation for all possible answers in *online mode* of the bot (e.g. when bot is running and answering questions from many users). This is the reason why you will create a *database* with pre-computed representations. These representations will be arranged by non-overlaping tags (programming languages), so that the search of the answer can be performed only within one tag each time. This will make our bot even more efficient and allow not to store all the database in RAM. 

Load StarSpace embeddings which were trained on Stack Overflow posts. These embeddings were trained in *supervised mode* for duplicates detection on the same corpus that is used in search. We can account on that these representations will allow us to find closely related answers for a question. 

If for some reasons you didn't train StarSpace embeddings in the assignment 3, you can use [pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google. All instructions about how to work with these vectors were provided in the same assignment. However, we highly recommend to use StarSpace's embeddings, because it contains more appropriate embeddings. If you chose to use Google's embeddings, delete the words, which are not in Stackoverflow data.

In [23]:
from utils import load_embeddings
starspace_embeddings, embeddings_dim = load_embeddings(data_dir+'starspace_embedding.tsv')

Since we want to precompute representations for all possible answers, we need to load the whole posts dataset, unlike we did for the intent classifier:

In [24]:
posts_df = pd.read_csv(data_dir+'tagged_posts.tsv', sep='\t')

Look at the distribution of posts for programming languages (tags) and find the most common ones. 
You might want to use pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [count](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.count.html) methods:

In [25]:
counts_by_tag = posts_df.groupby(['tag']).count()

In [26]:
counts_by_tag

,post_id,title
tag,,
c#,394451,394451
c_cpp,281300,281300
java,383456,383456
javascript,375867,375867
php,321752,321752
python,208607,208607
r,36359,36359
ruby,99930,99930
swift,34809,34809


In [27]:
counts_by_tag = posts_df['tag'].value_counts().to_dict()

counts_by_tag

{'c#': 394451,
 'java': 383456,
 'javascript': 375867,
 'php': 321752,
 'c_cpp': 281300,
 'python': 208607,
 'ruby': 99930,
 'r': 36359,
 'vb': 35044,
 'swift': 34809}

Now for each `tag` you need to create two data structures, which will serve as online search index:
* `tag_post_ids` — a list of post_ids with shape `(counts_by_tag[tag],)`. It will be needed to show the title and link to the thread;
* `tag_vectors` — a matrix with shape `(counts_by_tag[tag], embeddings_dim)` where embeddings for each answer are stored.

Implement the code which will calculate the mentioned structures and dump it to files. It should take several minutes to compute it.

In [28]:
import os

from utils import question_to_vec

os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts['post_id'].values
    
    tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(title, starspace_embeddings, embeddings_dim) 

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

## Part III. Putting all together

Now let's combine everything that we have done and enable the bot to maintain a dialogue. We will teach the bot to sequentially determine the intent and, depending on the intent, select the best answer. As soon as we do this, we will have the opportunity to chat with the bot and check how well it answers questions.

Implement Dialogue Manager that will generate the best answer. In order to do this, you should open *dialogue_manager.py* and fill in the gaps.

In [29]:
from dialogue_manager import DialogueManager

In [30]:
#import configparser
#config = configparser.ConfigParser();
#config.read('settings_secret.ini');

config={'PATH':RESOURCE_PATH,
        'TWITTER_API':'1234',
        'TELEGRAM':'abcd' }


In [31]:
dialogue_manager = DialogueManager(config) ######### YOUR CODE HERE #############

Loading resources...


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gshyam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/gshyam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gshyam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now we are ready to test our chat bot! Let's chat with the bot and ask it some questions. Check that the answers are reasonable.

In [33]:
questions = [
    "Hey", 
    "How are you doing?", 
    "What's your hobby?", 
    "How to write a loop in python?",
    "How to delete rows in pandas?",
    "python3 re",
    "What is the difference between c and c++",
    "Multithreading in Java",
    "Catch exceptions C++",
    "What is AI?",
]

In [35]:
for question in questions:
    #print (question.lower())
    answer = dialogue_manager.generate_answer(question)
    print('Q: %s\nA: %s \n' % (question, answer))

question Hey


AttributeError: 'ChatBot' object has no attribute 'predict'